<a href="https://colab.research.google.com/github/Arihant987/Rag_Langchain/blob/main/Rag_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index==0.9.45.post1 arize-phoenix==2.2.1 pyvis

INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.3

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.query_engine.pandas import PandasInstructionParser
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

In [87]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/train.csv'
df = pd.read_csv(file_path)
df.head(5)

,Question,Source,Category,Answer
0,what is the full name of the company,Customer Profile,Business,Duetto Research Inc
1,what is duetto's company website,Customer Profile,Business,https://www.duettocloud.com
2,what is the company address,Customer Profile,Business,"333 Bush Street, Floor 4\nSan Francisco, CA"
3,please provide a duetto company brief description,Customer Profile,Business,Hotel revenue management software & solutions
4,please provide a duetto company detailed descr...,Customer Profile,Business,"Cloud-based, scalable, and easy-to-use revenue..."


In [88]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python try to find answer or solution in df.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
llm = OpenAI(model="gpt-4")

In [89]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
qp.add_link("response_synthesis_prompt", "llm2")

In [90]:
df.isnull().sum()
df.fillna(0, inplace=True)

In [91]:
response = qp.run(
    query_str="Product list ?",
)

> Running module input with input: 
query_str: Product list ?

> Running module pandas_prompt with input: 
query_str: Product list ?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python try to find answer or solution in df.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
  Unnamed: 0        Unnamed: 1     ...

> Running module pandas_output_parser with input: 
input: assistant: df['Unnamed: 3'].unique()

> Running module response_synthesis_prompt with input: 
query_str: Product list ?
pandas_instructions: assistant: df['Unnamed: 3'].unique()
pandas_output: [0 'Theme' 'Algorithms, Analytics & Forecasting']

> Running module llm2 with input: 
messages: Given an input question, synthesize a response from the query results.
Query: Product list ?

Pandas Instructions (optional):
df['Unnamed: 3'].unique()

Pandas Output: [0 'Theme' 'Algorithms, Analytic...



In [ ]:
print(response.message.content)

In [ ]:
import pandas as pd
from pathlib import Path

data_dir = Path('/content/drive/My Drive/Train_folder/')
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: /content/drive/My Drive/Train_folder/Duetto - normalized.xlsx - Normalized.csv
processing file: /content/drive/My Drive/Train_folder/Duetto-RFP-Q&A v2.xlsx - Q&A.csv


In [ ]:
csv_files

[PosixPath('/content/drive/My Drive/Train_folder/Duetto - normalized.xlsx - Normalized.csv'),
 PosixPath('/content/drive/My Drive/Train_folder/Duetto-RFP-Q&A v2.xlsx - Q&A.csv')]

In [ ]:
tableinfo_dir = "TableInfo"
!mkdir {tableinfo_dir}

In [ ]:
from llama_index.program import LLMTextCompletionProgram
from llama_index.bridge.pydantic import BaseModel, Field
from llama_index.llms import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise.
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-4"),
    prompt_template_str=prompt_str,
)

In [ ]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

Processed table: duetto_company_profile
Processed table: analytics_forecasting_responses


In [ ]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re


# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: col for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

Creating table: duetto_company_profile
Creating table: analytics_forecasting_responses


In [ ]:
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [ ]:
from llama_index.retrievers import SQLRetriever
from typing import List
from llama_index.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

In [ ]:
from llama_index.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.prompts import PromptTemplate
from llama_index.query_pipeline import FnComponent
from llama_index.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


In [ ]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)
llm = OpenAI(model="gpt-4")

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [ ]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [ ]:
response = qp.run(
    query="What products do we duetto provide to customers ?"
)
print(str(response))

> Running module input with input: 
query: What products do we duetto provide to customers ?

> Running module table_retriever with input: 
input: What products do we duetto provide to customers ?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: What products do we duetto provide to customers ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can ord

In [ ]:
response = qp.run(query="Who are the top clients and why ?")
print(str(response))

> Running module input with input: 
query: Who are the top clients and why ?

> Running module table_retriever with input: 
input: Who are the top clients and why ?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: Who are the top clients and why ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return...

In [ ]:
response = qp.run(query="When was company established ?")
print(str(response))

> Running module input with input: 
query: When was company established ?

> Running module table_retriever with input: 
input: When was company established ?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: When was company established ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return...

> Runni

In [ ]:
from llama_index.query_pipeline import QueryPipeline as QP
from llama_index.service_context import ServiceContext

qp = QP(verbose=True)
# NOTE: service context will be deprecated in v0.10 (though will still be backwards compatible)
service_context = ServiceContext.from_defaults(callback_manager=qp.callback_manager)

In [ ]:
from llama_index import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.schema import TextNode
from llama_index.storage import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes, service_context=service_context)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index", service_context=service_context
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: analytics_forecasting_responses
Indexing rows in table: duetto_company_profile


In [ ]:
from llama_index.retrievers import SQLRetriever
from typing import List
from llama_index.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

In [ ]:
from llama_index.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp.add_modules({
    "input": InputComponent(),
    "table_retriever": obj_retriever,
    "table_output_parser": table_parser_component,
    "text2sql_prompt": text2sql_prompt,
    "text2sql_llm": llm,
    "sql_output_parser": sql_parser_component,
    "sql_retriever": sql_retriever,
    "response_synthesis_prompt": response_synthesis_prompt,
    "response_synthesis_llm": llm,
})

In [ ]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [ ]:
response = qp.run(
    query="When was company established ?"
)
print(str(response))

> Running module input with input: 
query: When was company established ?

> Running module table_retriever with input: 
input: When was company established ?

> Running module table_output_parser with input: 
query_str: When was company established ?
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: When was company established ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results 

In [ ]:
response = qp.run(
    query="What is customer retention rate ?"
)
print(str(response))

> Running module input with input: 
query: What is customer retention rate ?

> Running module table_retriever with input: 
input: What is customer retention rate ?

> Running module table_output_parser with input: 
query_str: What is customer retention rate ?
table_schema_objs: [SQLTableSchema(table_name='analytics_forecasting_responses', context_str='The table contains responses to a set of questions related to algorithms, analytics, and forecasting. The questions were answ...

> Running module text2sql_prompt with input: 
query_str: What is customer retention rate ?
schema: Table 'analytics_forecasting_responses' has columns: Unnamed: 0 (VARCHAR), Unnamed: 1 (VARCHAR), Unnamed: 2 (VARCHAR), Unnamed: 3 (VARCHAR), Unnamed: 4 (VARCHAR), Unnamed: 5 (VARCHAR), Unnamed: 6 (VAR...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order 

In [ ]:
response = qp.run(query="Who are the top clients and why ?")
print(str(response))

> Running module input with input: 
query: Who are the top clients and why ?

> Running module table_retriever with input: 
input: Who are the top clients and why ?

> Running module table_output_parser with input: 
query_str: Who are the top clients and why ?
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: Who are the top clients and why ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order 

In [ ]:
response = qp.run(query="Duetto product list ?")
print(str(response))

> Running module input with input: 
query: Duetto product list ?

> Running module table_retriever with input: 
input: Duetto product list ?

> Running module table_output_parser with input: 
query_str: Duetto product list ?
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: Duetto product list ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return...

>

In [ ]:
response = qp.run(query="The solution is manual or automated ?")
print(str(response))

> Running module input with input: 
query: The solution is manual or automated ?

> Running module table_retriever with input: 
input: The solution is manual or automated ?

> Running module table_output_parser with input: 
query_str: The solution is manual or automated ?
table_schema_objs: [SQLTableSchema(table_name='analytics_forecasting_responses', context_str='The table contains responses to a set of questions related to algorithms, analytics, and forecasting. The questions were answ...

> Running module text2sql_prompt with input: 
query_str: The solution is manual or automated ?
schema: Table 'analytics_forecasting_responses' has columns: Unnamed: 0 (VARCHAR), Unnamed: 1 (VARCHAR), Unnamed: 2 (VARCHAR), Unnamed: 3 (VARCHAR), Unnamed: 4 (VARCHAR), Unnamed: 5 (VARCHAR), Unnamed: 6 (VAR...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer

In [83]:
response = qp.run(query="Is there any client support ?")
print(str(response))

> Running module input with input: 
query: Is there any client support ?

> Running module table_retriever with input: 
input: Is there any client support ?

> Running module table_output_parser with input: 
query_str: Is there any client support ?
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: Is there any client support ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a

In [84]:
response = qp.run(query="Is there any mobile application ?")
print(str(response))

> Running module input with input: 
query: Is there any mobile application ?

> Running module table_retriever with input: 
input: Is there any mobile application ?

> Running module table_output_parser with input: 
query_str: Is there any mobile application ?
table_schema_objs: [SQLTableSchema(table_name='duetto_company_profile', context_str='The table contains detailed information about the company Duetto Research Inc, including its full name, website, address, brief and de...

> Running module text2sql_prompt with input: 
query_str: Is there any mobile application ?
schema: Table 'duetto_company_profile' has columns: Question (VARCHAR), Source (VARCHAR), Category (VARCHAR), Answer (VARCHAR), and foreign keys: . The table description is: The table contains detailed inform...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order 